# Prerequisite
You need to have a working local installation of Ansys

## Sections
### I. Transmon only: render into Ansys.
1. Run finite element eigenmode analysis. <br>
1. Plot fields and display them. <br>
1. Set up EPR junction dictionary. <br>
1. Run EPR analysis on single mode. <br>
1. Get qubit freq and anharmonicity. <br>
1. Calculate EPR of substrate.  <br>
1. (Extra: Calculate surface EPR.) <br>

### II. Resonator only: render into Ansys.
1. Run finite element eigenmode analysis. <br>
1. Plot fields and display them. <br>
1. Calculate EPR of substrate. <br>
    
### III. Transmon & resonator: render into Ansys.
1. Run finite element eigenmode analysis. <br>
1. Plot fields and display them. <br>
1. Set up EPR junction dictionary. <br>
1. Run EPR analysis on the two modes. <br>
1. Get qubit frequency and anharmonicity. <br>

In [1]:
%load_ext autoreload
%autoreload 2

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings, open_docs
import pyEPR as epr
from qiskit_metal.qlibrary.qubits.transmon_pocket_cl import TransmonPocketCL
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket

from qiskit_metal.qlibrary.interconnects.meandered import RouteMeander
from qiskit_metal.qlibrary.interconnects.pathfinder import RoutePathfinder

from qiskit_metal.qlibrary.passives.cap_three_fingers import CapThreeFingers
from qiskit_metal.qlibrary.passives.launchpad_wb import LaunchpadWirebond

In [2]:
%metal_heading Tutorial 1: Create a single transmon qubit and analyze it using pyEPR.

In [3]:
%metal_print Part 1. Finite Element Eigenmode Analysis

### Create the design in Metal
Setup a design of a given dimension. Dimensions will be respected in the design rendering. <br>
Note that the design size extends from the origin into the first quadrant.

In [4]:
design = metal.designs.DesignPlanar()
gui = metal.MetalGUI(design)


design.chips.main.size['size_x'] = '9mm'
design.chips.main.size['size_y'] = '3mm'
# wide cpw 
design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '9 um' 

design.overwrite_enabled = True
design.chips.main

#design = designs.DesignPlanar({}, True)
#design.chips.main.size['size_x'] = '9mm'
#design.chips.main.size['size_y'] = '6mm'
#hfss = design.renderers.hfss

#gui = MetalGUI(design)

{'material': 'silicon',
 'layer_start': '0',
 'layer_end': '2048',
 'size': {'center_x': '0.0mm',
  'center_y': '0.0mm',
  'center_z': '0.0mm',
  'size_x': '9mm',
  'size_y': '3mm',
  'size_z': '-750um',
  'sample_holder_top': '890um',
  'sample_holder_bottom': '1650um'}}

  self.draw()



In [5]:
## Basic operation
options = dict(
    pad_width = '450 um',
    pad_height= '100 um',
    pad_gap = '30um',
    
    pocket_height = '650 um',
    cl_pocket_edge = '180',
    connection_pads = dict(
        R = dict(loc_W=+1, loc_H=+1, pocket_rise = '50um', pad_cpw_shift = '8um', pad_gap = '30 um', pad_width = '93um'),
        L = dict(loc_W=-1, loc_H=-1, pocket_rise = '50um', pad_cpw_shift = '8um', pad_gap = '30 um', pad_width = '93um'),
        B = dict(loc_W=+1, loc_H=-1, pocket_rise = '50um', pad_cpw_shift = '8um', pad_gap = '30 um', pad_width = '93um')
  
    ))

Q1 = TransmonPocket(design,'Q1', options = dict(
        pos_x='-1.7mm',
        pos_y='+0mm',
        gds_cell_name = 'TOP',
        hfss_inductance = '11.8nH',
        q3d_inductance =  '11.8nH',
        **options))

Q2 = TransmonPocket(design,'Q2', options = dict(
        pos_x='+1.7mm',
        pos_y='+0mm',
        gds_cell_name = 'TOP',
        hfss_inductance = '10.9nH',
        q3d_inductance =  '10.9nH',
        orientation = '180',
        **options))

Q3 = TransmonPocket(design,'Q3', options = dict(
        pos_x='+0.0mm',
        pos_y='+1.5mm',
        gds_cell_name = 'TOP',
        hfss_inductance = '12.8nH',
        q3d_inductance =  '12.8nH',
        orientation = '270',
        **options))


gui.rebuild()
gui.autoscale()

  self.draw()



In [6]:
# Connect two pins with a CPW
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, asymmetry='0 um', flip=False, fillet='90um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        fillet=fillet,
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        total_length=length)
    myoptions.update(options)
    myoptions.meander.asymmetry = asymmetry
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

RouteMeander.get_template_options(design)


options = Dict(
        fillet = '80um',
        lead=Dict(
            start_straight='100um',
            end_straight='100um'),
        trace_gap='9um',
        trace_width='15um')


cpw1 = connect('cpw1', 'Q1', 'B', 'Q2', 'R', '8.4 mm', f'-200um')
cpw2 = connect('cpw2', 'Q2', 'B', 'Q3', 'R', '8.2 mm', f'-900um')
cpw3 = connect('cpw3', 'Q3', 'B', 'Q1', 'R', '8.0 mm', f'-300um')

gui.rebuild()
gui.autoscale()

In [7]:
# Capacitor for readout resonators (3-finger type)
cap_Q1 = CapThreeFingers(design, 'Cap_Q1', options = dict(pos_x = '-3.2 mm',
                                                          pos_y = '0.5mm',
                                                         orientation = '90',
                                                         finger_length = '40 um',
                                                         trace_width='15 um'))
cap_Q2 = CapThreeFingers(design, 'Cap_Q2', options = dict(pos_x = '+3.2 mm',
                                                          pos_y = '-0.5 mm',
                                                         orientation = '-90',
                                                         finger_length = '40 um',
                                                         trace_width='15 um'))
cap_Q3 = CapThreeFingers(design, 'Cap_Q3', options = dict(pos_x = '0.5 mm',
                                                          pos_y = '3.2mm',
                                                         orientation = '-90',
                                                         finger_length = '40 um',
                                                         trace_width='15 um'))


gui.rebuild()
gui.autoscale()

In [8]:
# Readout resonators (for Q1 and Q2)
RO_Q1 = RouteMeander(design, 'RO_Q1', 
                    options = dict(pin_inputs = Dict(
                            start_pin = Dict(component = 'Q1',pin = 'L'),
                              end_pin = Dict(component = 'Cap_Q1',pin = 'b')),
                    lead = Dict(start_straight = '100 um',
                                end_straight = '100 um'),
                    meander = Dict(asymmetry = '-50 um'),
                    fillet = '90 um',
                    total_length = '7.5 mm'))

RO_Q2 = RouteMeander(design, 'RO_Q2', 
                    options = dict(pin_inputs = Dict(
                            start_pin = Dict(component = 'Q2',pin = 'L'),
                              end_pin = Dict(component = 'Cap_Q2',pin = 'b')),
                    lead = Dict(start_straight = '100 um',
                                end_straight = '100 um'),
                    meander = Dict(asymmetry = '-50 um'),
                    fillet = '90 um',
                    total_length = '7.3 mm'))

RO_Q3 = RouteMeander(design, 'RO_Q3', 
                    options = dict(pin_inputs = Dict(
                            start_pin = Dict(component = 'Q3',pin = 'L'),
                              end_pin = Dict(component = 'Cap_Q3',pin = 'b')),
                    lead = Dict(start_straight = '400 um',
                                end_straight = '100 um'),
                    meander = Dict(asymmetry = '-50 um'),
                    fillet = '90 um',
                    total_length = '7.1 mm'))

gui.rebuild()
gui.autoscale()

### Establish a connection between Metal and Ansys
First you will need to open Ansys. <br>
You can either open Ansys manually, or by executing one of the lines in the following cell. <br>
open_ansys() will look by default for the 2020 R2 version of Ansys. You can easily reroute it to your Ansys of choice by providing the name of the environment variable that contains the path (path_env) or the path itself (path)

In [9]:
#hfss.open_ansys()   # this opens Ansys 2020 R2 if present
# hfss.open_ansys(path_var='ANSYSEM_ROOT211')
# hfss.open_ansys(path='C:\Program Files\AnsysEM\AnsysEM20.2\Win64')
# hfss.open_ansys(path='../../../Program Files/AnsysEM/AnsysEM20.2/Win64')
hfss = design.renderers.hfss
hfss.open_ansys(path='C:\Program Files\AnsysEM\AnsysEM21.1\Win64')

If a project was not automatically opened, you can manually create/open one from the Ansys GUI. Alternatively you can execute the following cell.

In [ ]:
# hfss.new_ansys_project()

Finally connect this Jupyter Notebook session with the active Ansys project.

In [14]:
hfss.connect_ansys()
# hfss.connect_ansys('C:\\project_path\\', 'Project1')  # will open a saved project before linking the Jupyter session
hfss.add_eigenmode_design("TransmonQubit")

INFO 01:56PM [connect_project]: Connecting to Ansys Desktop API...
INFO 01:56PM [load_ansys_project]: 	Opened Ansys App
INFO 01:56PM [load_ansys_project]: 	Opened Ansys Desktop v2021.1.0
INFO 01:56PM [load_ansys_project]: 	Ansys Project was not found.
	 Project is None.
INFO 01:56PM [connect]: 	Connection to Ansys NOT established.  

INFO 01:56PM [connect]: 	 Project not detected in Ansys. Is there a project in your desktop app? 🤔 

INFO 01:56PM [connect]: 	 Design not detected in project. Is there a design in your project? 🤔 



AttributeError: 'NoneType' object has no attribute 'new_em_design'

### Prepare and run the analysis

Create and activate an eigenmode design called "TransmonQubit".

In [ ]:
hfss.add_eigenmode_design("TransmonQubit")

Render the single qubit in Metal, called Q1, to "TransmonQubit" design in Ansys.

In [ ]:
hfss.render_design([], [])
hfss.save_screenshot()

Set the convergence parameters and junction properties in the Ansys design. <br>
Then run the analysis and plot the convergence.

In [ ]:
# Analysis properties
setup = hfss.pinfo.setup
setup.passes = 10
print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

pinfo = hfss.pinfo
pinfo.design.set_variable('Lj', '10 nH')
pinfo.design.set_variable('Cj', '0 fF')
setup.analyze()

hfss.plot_convergences()

### Plot fields and display them
Display the Ansys modeler window and plot the E-field on the chip's surface.

In [ ]:
hfss.modeler._modeler.ShowWindow()
hfss.plot_ansys_fields('main')
hfss.save_screenshot()

Delete the newly created E-field plot to prepare for the next phase.

In [ ]:
hfss.plot_ansys_delete(['Mag_E1'])

In [ ]:
%metal_print Part 2. Energy-Participation-Ratio Analysis

### Set up EPR parameters
Specify the junctions in the model; in this case there's only one, namely 'jj'.

In [ ]:
#Non-linear (Josephson) junction
pinfo = hfss.pinfo # Project info
pinfo.junctions['jj'] = {'Lj_variable': 'Lj', 'rect': 'JJ_rect_Lj_Q1_rect_jj', 
                             'line': 'JJ_Lj_Q1_rect_jj_',  'Cj_variable': 'Cj'}
pinfo.validate_junction_info() # Check that valid names of variables and objects have been supplied
pinfo.dissipative['dielectrics_bulk'] = ['main'] # Dissipative elements: specify
# Handles  microwave analysis on eigenmode solutions
eprd = epr.DistributedAnalysis(pinfo)

Find the electric and magnetic energy stored in the substrate and the system as a whole.

In [ ]:
ℰ_elec = eprd.calc_energy_electric()
ℰ_elec_substrate = eprd.calc_energy_electric(None, 'main')
ℰ_mag = eprd.calc_energy_magnetic()

print(f"""
ℰ_elec_all       = {ℰ_elec}
ℰ_elec_substrate = {ℰ_elec_substrate}
EPR of substrate = {ℰ_elec_substrate / ℰ_elec * 100 :.1f}%

ℰ_mag    = {ℰ_mag}
""")

### Run EPR analysis

Perform EPR analysis for all modes and variations.

In [ ]:
eprd.do_EPR_analysis()

# 4a. Perform Hamiltonian spectrum post-analysis, building on mw solutions using EPR
epra = epr.QuantumAnalysis(eprd.data_filename)
epra.analyze_all_variations(cos_trunc = 8, fock_trunc = 7)

# 4b. Report solved results
swp_variable = 'Lj' # suppose we swept an optimetric analysis vs. inductance Lj_alice
epra.plot_hamiltonian_results(swp_variable=swp_variable)
epra.report_results(swp_variable=swp_variable, numeric=True)

Release Ansys session

In [ ]:
hfss.modeler._modeler.ShowWindow()
hfss.plot_ansys_fields('main')
hfss.save_screenshot()

In [ ]:
hfss.disconnect_ansys()

In [ ]:
%metal_heading Tutorial 2: Create and perform EPR analysis on a CPW resonator.

In [ ]:
%metal_print Part 1. Finite Element Eigenmode Analysis

#### Update the design in Metal
Connect the transmon to a CPW. <br>
The other end of the CPW connects to an open to ground termination.

In [ ]:
from qiskit_metal.qlibrary.connectors.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.interconnects.meandered import RouteMeander
otg = OpenToGround(design, 'open_to_ground', options=dict(pos_x='1.75mm',  pos_y='0um', orientation='0'))
RouteMeander(design, 'readout',  Dict(
        total_length='6 mm',
        fillet='90 um',
        lead = dict(start_straight='100um'),
        pin_inputs=Dict(
            start_pin=Dict(component='Q1', pin='readout'),
            end_pin=Dict(component='open_to_ground', pin='open')), ))

gui.rebuild()
gui.autoscale()

#### Render the resonator Design in Ansys
Connect again to Ansys. <br>
Then create and activate a separate eigenmode design called "Readout".

In [ ]:
hfss.connect_ansys()
hfss.add_eigenmode_design("Readout")

Render the readout and an open to ground termination. <br>
Add open endcaps to both ends of the readout.

In [ ]:
hfss.render_design(['readout', 'open_to_ground'], [('readout', 'start'), ('readout', 'end')])
hfss.save_screenshot()

#### Prepare and run the analysis, then plot E-field

Set variables in the Ansys design.

In [ ]:
# Analysis properties
setup = hfss.pinfo.setup
setup.passes = 10
print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

pinfo = hfss.pinfo
pinfo.design.set_variable('Lj', '10 nH')
pinfo.design.set_variable('Cj', '0 fF')
setup.analyze()

hfss.plot_convergences()

Recover eigenmode frequencies for each variation.

In [ ]:
eprd = epr.DistributedAnalysis(hfss.pinfo)
eprd.get_ansys_frequencies_all()

Display the Ansys modeler window and plot the E-field on the chip's surface.

In [ ]:
hfss.modeler._modeler.ShowWindow()
hfss.plot_ansys_fields('main')
hfss.save_screenshot()

Delete the newly created E-field plot.

In [ ]:
hfss.plot_ansys_delete(['Mag_E1'])

#### Try again with a larger number of steps (better convergence)

Update the number of passes, analyze, and plot convergence data.

In [ ]:
setup.passes = 15
setup.analyze()
hfss.plot_convergences()

Display the Ansys modeler window again and plot the E-field on the chip's surface with this updated number of passes. <br>
Note that the bright areas have become much smoother compared to the previous image, indicating better convergence.

In [ ]:
hfss.modeler._modeler.ShowWindow()
hfss.plot_ansys_fields('main')
hfss.save_screenshot()

In [ ]:
%metal_print Part 2. Energy-Participation-Ratio Analysis

Find the electric and magnetic energy stored in the substrate and the system as a whole.

In [ ]:
ℰ_elec = eprd.calc_energy_electric()
ℰ_elec_substrate = eprd.calc_energy_electric(None, 'main')
ℰ_mag = eprd.calc_energy_magnetic()

print(f"""
ℰ_elec_all       = {ℰ_elec}
ℰ_elec_substrate = {ℰ_elec_substrate}
EPR of substrate = {ℰ_elec_substrate / ℰ_elec * 100 :.1f}%

ℰ_mag_all       = {ℰ_mag}
ℰ_mag % of ℰ_elec_all  = {ℰ_mag / ℰ_elec * 100 :.1f}%
""")

Release Ansys session

In [ ]:
hfss.disconnect_ansys()

In [ ]:
%metal_heading Tutorial 3: Analyze the combined transmon + CPW resonator system. 

In [ ]:
%metal_print Part 1. Finite Element Eigenmode Analysis

Connect to Ansys, then create and activate an eigenmode Ansys design called "TransmonResonator".

In [ ]:
hfss.connect_ansys()
hfss.add_eigenmode_design("TransmonResonator")

Render everything in the Metal design, including both the transmon and the readout. <br>
Add an open endcap to the readout.

In [ ]:
hfss.render_design(['Q1', 'readout', 'open_to_ground'], [('readout', 'end')])
hfss.save_screenshot()

Set variables in the Ansys design. Note that this time we seek 2 eigenmodes - one with stronger fields near the transmon, the other with stronger fields near the resonator.

In [ ]:
# Analysis properties
setup = hfss.pinfo.setup
setup.n_modes = 2
setup.passes = 10
print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

pinfo = hfss.pinfo
pinfo.design.set_variable('Lj', '10 nH')
pinfo.design.set_variable('Cj', '0 fF')
setup.analyze()

hfss.plot_convergences()

Display the Ansys modeler window again and plot the E-field on the chip's surface.

In [ ]:
hfss.modeler._modeler.ShowWindow()
hfss.plot_ansys_fields('main')
hfss.save_screenshot()

Delete the newly created E-field plot.

In [ ]:
hfss.plot_ansys_delete(['Mag_E1'])

In [ ]:
%metal_print Part 2. Energy-Participation-Ratio Analysis

Specify the junctions in the model; in this case there's only one, namely 'jj'.

In [ ]:
#Non-linear (Josephson) junction
pinfo = hfss.pinfo 
pinfo.junctions['jj'] = {'Lj_variable': 'Lj',  'rect': 'JJ_rect_Lj_Q1_rect_jj', 
                             'line': 'JJ_Lj_Q1_rect_jj_',  'Cj_variable': 'Cj'}
pinfo.validate_junction_info() # Check that valid names of variables and objects have been supplied
pinfo.dissipative['dielectrics_bulk'] = ['main'] # Dissipative elements: specify
# Handles  microwave analysis on eigenmode solutions
eprd = epr.DistributedAnalysis(pinfo)

In [ ]:
eprd = epr.DistributedAnalysis(pinfo)
eprd.set_mode(1)

Find the electric and magnetic energy stored in the substrate and the system as a whole.

In [ ]:
ℰ_elec = eprd.calc_energy_electric()
ℰ_elec_substrate = eprd.calc_energy_electric(None, 'main')
ℰ_mag = eprd.calc_energy_magnetic()

print(f"""
ℰ_elec_all       = {ℰ_elec}
ℰ_elec_substrate = {ℰ_elec_substrate}
EPR of substrate = {ℰ_elec_substrate / ℰ_elec * 100 :.1f}%

ℰ_mag_all       = {ℰ_mag}
ℰ_mag % of ℰ_elec_all  = {ℰ_mag / ℰ_elec * 100 :.1f}%
""")

In [ ]:
eprd.set_mode(0)

Perform EPR analysis for all modes and variations.

In [ ]:
eprd.do_EPR_analysis()

# 4a. Perform Hamiltonian spectrum post-analysis, building on mw solutions using EPR
epra = epr.QuantumAnalysis(eprd.data_filename)
epra.analyze_all_variations(cos_trunc = 8, fock_trunc = 7)

# 4b. Report solved results
swp_variable = 'Lj' # suppose we swept an optimetric analysis vs. inductance Lj_alice
epra.plot_hamiltonian_results(swp_variable=swp_variable)
epra.report_results(swp_variable=swp_variable, numeric=True)

Release Ansys's session

In [ ]:
hfss.disconnect_ansys()

In [ ]:
%metal_heading Tutorial 4: Analyze a coupled 2 transmon system. 

In [ ]:
%metal_print Part 1. Finite Element Eigenmode Analysis

Connect to Ansys, then create and activate an eigenmode Ansys design called "TransmonResonator".

In [ ]:
hfss.connect_ansys()
hfss.add_eigenmode_design("TwoTransmon")

Create a Metal design consisting of 2 transmons connected by a short coupler.

In [ ]:
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.interconnects.straight_path import RouteStraight

design.delete_all_components()

q1 = TransmonPocket(design, 'Q1', options = dict(
    pad_width = '425 um', 
    pocket_height = '650um',
    connection_pads=dict(
        readout = dict(loc_W=+1,loc_H=+1, pad_width='200um')
    )))

q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x = '1.0 mm',
    pad_width = '425 um', 
    pocket_height = '650um',
    connection_pads=dict(
        readout = dict(loc_W=-1,loc_H=+1, pad_width='200um')
    )))

coupler = RouteStraight(design, 'coupler', Dict(
        pin_inputs=Dict(
            start_pin=Dict(component='Q1', pin='readout'),
            end_pin=Dict(component='Q2', pin='readout')), ))

gui.rebuild()
gui.autoscale()

Change inductance value for one of the transmons.

In [ ]:
design.components['Q2'].options['hfss_inductance'] = '12nH'
print(design.components['Q1'].options['hfss_inductance'])
print(design.components['Q2'].options['hfss_inductance'])

Render everything in the Metal design, including both transmons and the coupler.

In [ ]:
hfss.render_design(['Q1', 'coupler', 'Q2'], [])
hfss.save_screenshot()

Set variables in the Ansys design. As before, we seek 2 modes.

In [ ]:
# Analysis properties
setup = hfss.pinfo.setup
setup.n_modes = 2
setup.passes = 10
print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

pinfo = hfss.pinfo
pinfo.design.set_variable('Lj', '10 nH')
pinfo.design.set_variable('Cj', '0 fF')
setup.analyze()

hfss.plot_convergences()

(Optional) To perform a sweep of Lj, set the hfss inductance for Q1 to be Lj instead of 10 nH in its renderer options as shown in the following cell.

In [ ]:
design.components['Q1'].options['hfss_inductance'] = 'Lj'
pinfo.design.set_variable('Lj', '10 nH')
pinfo.design.set_variable('Cj', '0 fF')
hfss.render_design(['Q1', 'coupler', 'Q2'], [])

Display the Ansys modeler window again and plot the E-field on the chip's surface.

In [ ]:
hfss.modeler._modeler.ShowWindow()
hfss.plot_ansys_fields('main')
hfss.save_screenshot()

Delete the newly created E-field plot.

In [ ]:
hfss.plot_ansys_delete(['Mag_E1'])

In [ ]:
%metal_print Part 2. Energy-Participation-Ratio Analysis

Specify the junctions in the model; in this case there are 2 junctions, namely 'jj' and 'jj2'.

In [ ]:
#Non-linear (Josephson) junction
pinfo = hfss.pinfo 
pinfo.junctions['jj'] = {'Lj_variable': 'Lj',  'rect': 'JJ_rect_Lj_Q1_rect_jj', 
                             'line': 'JJ_Lj_Q1_rect_jj_',  'Cj_variable': 'Cj'}
pinfo.junctions['jj2'] = {'Lj_variable': 'Lj',  'rect': 'JJ_rect_Lj_Q2_rect_jj', 
                             'line': 'JJ_Lj_Q2_rect_jj_',  'Cj_variable': 'Cj'}
pinfo.validate_junction_info() # Check that valid names of variables and objects have been supplied
pinfo.dissipative['dielectrics_bulk'] = ['main'] # Dissipative elements: specify
# Handles  microwave analysis on eigenmode solutions
eprd = epr.DistributedAnalysis(pinfo)

Find the electric and magnetic energy stored in the substrate and the system as a whole.

In [ ]:
ℰ_elec = eprd.calc_energy_electric()
ℰ_elec_substrate = eprd.calc_energy_electric(None, 'main')
ℰ_mag = eprd.calc_energy_magnetic()

print(f"""
ℰ_elec_all       = {ℰ_elec}
ℰ_elec_substrate = {ℰ_elec_substrate}
EPR of substrate = {ℰ_elec_substrate / ℰ_elec * 100 :.1f}%

ℰ_mag_all       = {ℰ_mag}
ℰ_mag % of ℰ_elec_all  = {ℰ_mag / ℰ_elec * 100 :.1f}%
""")

Perform EPR analysis for all modes and variations.

In [ ]:
eprd.do_EPR_analysis()

# 4a. Perform Hamiltonian spectrum post-analysis, building on mw solutions using EPR
epra = epr.QuantumAnalysis(eprd.data_filename)
epra.analyze_all_variations(cos_trunc = 8, fock_trunc = 7)

# 4b. Report solved results
swp_variable = 'Lj' # suppose we swept an optimetric analysis vs. inductance Lj_alice
epra.plot_hamiltonian_results(swp_variable=swp_variable)
epra.report_results(swp_variable=swp_variable, numeric=True)

Release Ansys's session

In [ ]:
hfss.disconnect_ansys()